# Protein Sequence Representation Model

TorchProtein定义了多种基于序列的模型来学习蛋白质序列表示。在本教程中，我们使用一个两层的一维卷积神经网络（1D CNN）作为所有考虑任务的蛋白质序列表示模型。首先，让我们通过models.ProteinCNN模块来定义这样一个模型。

In [1]:
from torchdrug import models

model = models.ProteinCNN(input_dim=21,
                          hidden_dims=[1024, 1024],
                          kernel_size=5, padding=2, readout="max")

# Task 1: Protein-wise Property Prediction

我们希望解决的第一类任务是预测蛋白质的整体性质。我们以β-内酰胺酶活性预测任务为例，该任务旨在预测对TEM-1 β-内酰胺酶蛋白的突变效应。

在定义数据集之前，我们首先需要定义我们要对蛋白质执行的转换操作。我们考虑两种转换操作：（1）为了降低基于序列的模型的内存成本，常见做法是截断过长的蛋白质序列。在TorchProtein中，我们可以通过指定最大长度（max_length参数）和截断位置（随机残基或第一个残基）（random参数）来定义蛋白质截断转换。 （2）此外，由于我们希望将残基特征作为序列模型的节点特征使用，我们还需要定义蛋白质的视图转换。在数据集构建过程中，我们可以将两个转换的组合作为参数传递。


In [2]:
from torchdrug import transforms

truncate_transform = transforms.TruncateProtein(max_length=200, random=False)
protein_view_transform = transforms.ProteinView(view="residue")
transform = transforms.Compose([truncate_transform, protein_view_transform])

然后，我们通过datasets.BetaLactamase构建数据集，该数据集文件将自动下载。在这个数据集中，每个样本的标签是一个实数，表示蛋白质的适应性值。通过打开residue_only选项，TorchProtein将使用data.Protein.from_sequence_fast来加载蛋白质，从而提高加载速度。我们可以通过split()方法获取预定义的训练、验证和测试集划分。

In [3]:
from torchdrug import datasets

dataset = datasets.BetaLactamase("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=transform)
train_set, valid_set, test_set = dataset.split()
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

18:53:58   Extracting /home/weibin/protein-datasets/beta_lactamase.tar.gz to /home/weibin/protein-datasets


Constructing proteins from sequences: 100%|██████████| 5198/5198 [00:10<00:00, 518.35it/s]


The label of first sample:  0.9426838159561157
train samples: 4158, valid samples: 520, test samples: 520


为了进行β-内酰胺酶活性预测，我们将CNN编码器包装在tasks.PropertyPrediction模块中，该模块在CNN之上添加了一个特定任务的MLP预测头。

In [4]:
from torchdrug import tasks

task = tasks.PropertyPrediction(model, task=dataset.tasks,
                                criterion="mse", metric=("mae", "rmse", "spearmanr"),
                                normalization=False, num_mlp_layer=2)

现在我们可以训练我们的模型了。我们为模型设置了一个优化器，并将所有内容放在一个Engine实例中。在这个任务上，我们的模型训练10个epochs大约需要2分钟的时间。最后，我们评估模型在验证集上的性能。

In [5]:
import torch
from torchdrug import core

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=64)
solver.train(num_epoch=10)
solver.evaluate("valid")

18:59:35   Preprocess training set
18:59:49   {'batch_size': 64,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.PropertyPrediction',
          'criterion': 'mse',
          'graph_construction_model': None,
          'metric': ('mae', 'rmse', 'spearmanr'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
                    'short_cut': False,
                    'stride

{'mean absolute error [scaled_effect1]': tensor(0.2937, device='cuda:0'),
 'root mean squared error [scaled_effect1]': tensor(0.3231, device='cuda:0'),
 'spearmanr [scaled_effect1]': tensor(0.4503, device='cuda:0')}

# Task 2: Residue-wise Property Prediction

我们考虑的第二类任务是预测残基级的性质。我们以二级结构预测任务为例。

我们首先通过datasets.SecondaryStructure构建数据集，在其中我们使用cb513测试集。对于残基级的任务，我们通常保留蛋白质的整个序列，所以这里我们只使用ProteinView转换。目标字段表示每个残基的二级结构（卷曲、链或螺旋），掩码字段表示每个二级结构标签是否有效。这两个字段的长度与蛋白质序列的长度相同。

In [6]:
dataset = datasets.SecondaryStructure("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=protein_view_transform)
train_set, valid_set, test_set = dataset.split(["train", "valid", "cb513"])
print("SS3 label: ", dataset[0]["graph"].target[:10])
print("Valid mask: ", dataset[0]["graph"].mask[:10])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:03:47   Extracting /home/weibin/protein-datasets/secondary_structure.tar.gz to /home/weibin/protein-datasets


Constructing proteins from sequences:   0%|          | 0/11497 [00:00<?, ?it/s]/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/data/protein.py:261: UserWarning: Unknown residue symbol `X`. Treat as glycine
  warnings.warn("Unknown residue symbol `%s`. Treat as glycine" % residue)
Constructing proteins from sequences: 100%|██████████| 11497/11497 [00:20<00:00, 549.54it/s]

SS3 label:  tensor([2, 2, 2, 0, 0, 0, 0, 0, 2, 2])
Valid mask:  tensor([True, True, True, True, True, True, True, True, True, True])
train samples: 8678, valid samples: 2170, test samples: 513


为了进行二级结构预测，我们将CNN编码器包装在tasks.NodePropertyPrediction模块中，该模块在CNN之上添加了一个特定任务的MLP预测头。

In [7]:
task = tasks.NodePropertyPrediction(model, criterion="ce",
                                    metric=("micro_acc", "macro_acc"),
                                    num_mlp_layer=2, num_class=3)

我们对模型进行了5个epochs的训练，大约需要5分钟的时间，最后在验证集上进行了评估。

In [8]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=128)
solver.train(num_epoch=5)
solver.evaluate("valid")

19:05:15   Preprocess training set
19:05:16   {'batch_size': 128,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.NodePropertyPrediction',
          'criterion': 'ce',
          'metric': ('micro_acc', 'macro_acc'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
                    'short_cut': False,
                    'stride': 1},
          'normalization': True,
  

{'micro_acc': tensor(0.6443, device='cuda:0'),
 'macro_acc': tensor(0.6499, device='cuda:0')}

# Task 3: Contact Prediction

我们要解决的第三个任务是预测在折叠结构中是否存在任意两个残基之间的接触，即进行接触预测。

我们首先通过datasets.ProteinNet构建数据集。residue_position字段表示每个残基的三维坐标，mask字段表示每个残基位置是否有效。这两个字段的长度与蛋白质序列的长度相同。

In [9]:
dataset = datasets.ProteinNet("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=protein_view_transform)
train_set, valid_set, test_set = dataset.split()
print("Residue position: ", dataset[0]["graph"].residue_position[:3])
print("Valid mask: ", dataset[0]["graph"].mask[:3])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:08:51   Extracting /home/weibin/protein-datasets/proteinnet.tar.gz to /home/weibin/protein-datasets


Constructing proteins from sequences: 100%|██████████| 25557/25557 [00:43<00:00, 589.49it/s]

Residue position:  tensor([[ 2.0940e+00,  2.0000e-03, -1.2420e+00],
        [ 5.1260e+00, -2.0210e+00, -2.3290e+00],
        [ 7.5230e+00,  6.1500e-01, -3.6610e+00]])
Valid mask:  tensor([True, True, True])
train samples: 25299, valid samples: 224, test samples: 34


为了进行接触预测，我们将CNN编码器包装在tasks.ContactPrediction模块中，该模块在CNN之上添加了一个特定任务的MLP预测头。如果两个残基之间的序列间隔大于gap，并且它们的欧氏距离在阈值内，则认为它们发生了相互作用。与之前的任务不同，现在任务中定义了最大截断长度max_length，因为接触预测任务中测试集上的截断行为不同。对于测试集，为了节省内存，我们将测试序列根据max_length分成几个块。

In [10]:
task = tasks.ContactPrediction(model, max_length=500, random_truncate=True, threshold=8.0, gap=6,
                               criterion="bce", metric=("accuracy", "prec@L5", "prec@5"), num_mlp_layer=2)

由于原始训练集包含大量样本，而在这个任务中只能使用较小的批量大小，因此我们使用包含1000个样本的原始训练集的子集进行训练。我们对模型进行了1个epoch的训练，大约需要4分钟的时间，最后在验证集上进行了评估。

In [14]:
from torch.utils import data as torch_data

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
sub_train_set = torch_data.random_split(train_set, [1000, len(train_set) - 1000])[0]
solver = core.Engine(task, sub_train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=1)
solver.train(num_epoch=1)
solver.evaluate("valid")

19:13:49   Preprocess training set
19:13:49   {'batch_size': 1,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.ContactPrediction',
          'criterion': 'bce',
          'gap': 6,
          'max_length': 500,
          'metric': ('accuracy', 'prec@L5', 'prec@5'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
                    'short_cut': False,
                   

RuntimeError: CUDA out of memory. Tried to allocate 208.00 MiB (GPU 0; 23.87 GiB total capacity; 9.24 GiB already allocated; 76.94 MiB free; 9.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Task 4: Protein-Protein Interaction (PPI) Prediction

我们考虑的第四个任务是预测两个相互作用蛋白质的结合亲和力，即进行PPI亲和力预测。

我们首先通过datasets.PPIAffinity构建数据集，其中每个样本是一对蛋白质，并且与一个连续标签相关联，表示结合亲和力。由于我们现在需要对两个蛋白质进行转换，因此我们需要在转换函数中指定键值。

In [15]:
truncate_transform_ = transforms.TruncateProtein(max_length=200, keys=("graph1", "graph2"))
protein_view_transform_ = transforms.ProteinView(view="residue", keys=("graph1", "graph2"))
transform_ = transforms.Compose([truncate_transform_, protein_view_transform_])
dataset = datasets.PPIAffinity("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=transform_)
train_set, valid_set, test_set = dataset.split()
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:14:32   Extracting /home/weibin/protein-datasets/ppi_affinity.zip to /home/weibin/protein-datasets


Constructing proteins from sequences: 100%|██████████| 2950/2950 [00:09<00:00, 315.76it/s]

The label of first sample:  -12.2937
train samples: 2421, valid samples: 203, test samples: 326


为了进行PPI亲和力预测，我们将CNN编码器包装在tasks.InteractionPrediction模块中，该模块在CNN之上添加了一个特定任务的MLP预测头。

In [16]:
task = tasks.InteractionPrediction(model, task=dataset.tasks,
                                   criterion="mse", metric=("mae", "rmse", "spearmanr"),
                                   normalization=False, num_mlp_layer=2)

我们将模型训练10个epoch，大约需要2分钟的时间，最后在验证集上进行评估。

In [17]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=64)
solver.train(num_epoch=10)
solver.evaluate("valid")

19:14:42   Preprocess training set
19:14:47   {'batch_size': 64,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.InteractionPrediction',
          'criterion': 'mse',
          'graph_construction_model': None,
          'metric': ('mae', 'rmse', 'spearmanr'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
                    'short_cut': False,
                    'str

{'mean absolute error [interaction]': tensor(2.1606, device='cuda:0'),
 'root mean squared error [interaction]': tensor(2.8118, device='cuda:0'),
 'spearmanr [interaction]': tensor(0.5017, device='cuda:0')}

# Task 5: Protein-Ligand Interaction (PLI) Prediction

我们考虑的第五个任务是预测蛋白质和小分子（即配体）的结合亲和力。我们以BindingDB上的PLI预测为例。

我们首先通过datasets.BindingDB构建数据集，其中每个样本是一对蛋白质和配体，并且与一个连续标签相关联，表示结合亲和力。我们使用holdout_test集作为测试集。

In [18]:
truncate_transform_ = transforms.TruncateProtein(max_length=200, keys="graph1")
protein_view_transform_ = transforms.ProteinView(view="residue", keys="graph1")
transform_ = transforms.Compose([truncate_transform_, protein_view_transform_])
dataset = datasets.BindingDB("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=transform_)
train_set, valid_set, test_set = dataset.split(["train", "valid", "holdout_test"])
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

19:16:05   Extracting /home/weibin/protein-datasets/BindingDB_Kd.tar.gz to /home/weibin/protein-datasets


Constructing proteins from sequences:  29%|██▊       | 5091/17819 [00:32<01:14, 169.92it/s]/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Na`
  warnings.warn("Unknown value `%s`" % x)
Constructing proteins from sequences:  31%|███       | 5562/17819 [00:34<01:12, 168.72it/s]/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
Constructing proteins from sequences:  68%|██████▊   | 12186/17819 [01:16<00:30, 181.78it/s]/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown value `Mn`
  warnings.warn("Unknown value `%s`" % x)
Constructing proteins from sequences:  74%|███████▍  | 13262/17819 [01:22<00:26, 174.46it/s]/home/weibin/miniconda3/envs/torchdrug/lib/python3.8/site-packages/torchdrug/data/feature.py:42: UserWarning: Unknown val

The label of first sample:  5.823908740944319
train samples: 7900, valid samples: 878, test samples: 5230


为了进行PLI预测，我们需要一个额外的配体图编码器来提取配体的表示。我们将一个4层的图同构网络（GIN）定义为配体图编码器。然后，我们将CNN编码器和GIN编码器包装在tasks.InteractionPrediction模块中，该模块在CNN和GIN之上添加了一个特定任务的MLP预测头。

In [19]:
model2 = models.GIN(input_dim=66,
                    hidden_dims=[256, 256, 256, 256],
                    batch_norm=True, short_cut=True, concat_hidden=True)

task = tasks.InteractionPrediction(model, model2=model2, task=dataset.tasks,
                                   criterion="mse", metric=("mae", "rmse", "spearmanr"),
                                   normalization=False, num_mlp_layer=2)

我们将模型训练5个epoch，大约需要3分钟的时间，最后在验证集上进行评估。

In [20]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=16)
solver.train(num_epoch=5)
solver.evaluate("valid")

19:18:00   Preprocess training set
19:18:15   {'batch_size': 16,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.InteractionPrediction',
          'criterion': 'mse',
          'graph_construction_model': None,
          'metric': ('mae', 'rmse', 'spearmanr'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
                    'short_cut': False,
                    'str

RuntimeError: mat1 and mat2 shapes cannot be multiplied (482x67 and 66x256)